In [1]:
!pip install --quiet optuna

     |████████████████████████████████| 348 kB 7.2 MB/s 
     |████████████████████████████████| 81 kB 6.4 MB/s 
     |████████████████████████████████| 209 kB 15.5 MB/s 
     |████████████████████████████████| 78 kB 6.7 MB/s 
     |████████████████████████████████| 112 kB 16.2 MB/s 
     |████████████████████████████████| 49 kB 6.0 MB/s 
     |████████████████████████████████| 147 kB 15.6 MB/s 


In [2]:
import numpy as np
import pandas as pd
import optuna as opt
import xgboost as xgb
from optuna.samplers import TPESampler
from sklearn.preprocessing import StandardScaler
from pathlib import Path
import gc

from sklearn.metrics import mean_absolute_error, mean_squared_error

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
RANDOM_SEED = 2
DATA_DIR = Path("/content/drive/MyDrive/CS760")

In [5]:
df_train = pd.read_parquet(DATA_DIR/"train_main.parquet.snappy")
df_val = pd.read_parquet(DATA_DIR/"val_main.parquet.snappy")
df_test = pd.read_parquet(DATA_DIR/"test_main.parquet.snappy")

print(f"Shape of the training data : {df_train.shape}")
print(f"Shape of the validation data : {df_val.shape}")
print(f"Shape of the test data : {df_test.shape}")

Shape of the training data : (2060626, 13)
Shape of the validation data : (257578, 13)
Shape of the test data : (257579, 13)


In [6]:
df_train.describe().T

,count,mean,std,min,25%,50%,75%,max
r_id,2060626.0,3.522730e+06,2.016756e+06,4.000000e+00,1.803866e+06,3.483320e+06,5.288280e+06,6.990278e+06
r_stars,2060626.0,3.607754e+00,1.538412e+00,1.000000e+00,2.000000e+00,4.000000e+00,5.000000e+00,5.000000e+00
r_stars_square,2060626.0,1.538260e+01,9.617924e+00,1.000000e+00,4.000000e+00,1.600000e+01,2.500000e+01,2.500000e+01
r_length,2060626.0,1.332717e+02,1.149773e+02,0.000000e+00,5.700000e+01,1.010000e+02,1.710000e+02,1.061000e+03
u_friends_count,2060626.0,1.888371e+02,5.663421e+02,1.000000e+00,2.000000e+00,4.000000e+01,1.680000e+02,1.499500e+04
u_review_count,2060626.0,1.911808e+02,4.966023e+02,0.000000e+00,1.100000e+01,4.400000e+01,1.760000e+02,1.747300e+04
u_month_age,2060626.0,4.274377e+01,3.454640e+01,3.802649e-07,1.364517e+01,3.644688e+01,6.515015e+01,2.018375e+02
b_stars,2060626.0,3.742639e+00,7.950195e-01,1.000000e+00,3.500000e+00,4.000000e+00,4.500000e+00,5.000000e+00
b_review_count,2060626.0,3.371921e+02,6.668911e+02,5.000000e+00,4.000000e+01,1.250000e+02,3.410000e+02,7.568000e+03
r_sen,2060626.0,1.823936e-01,1.926882e-01,-1.000000e+00,6.007800e-02,1.808695e-01,3.004768e-01,1.000000e+00


In [7]:
X_train, y_train = df_train.drop(['r_useful', 'r_id'], axis=1).values, df_train['r_useful'].values
X_val, y_val = df_val.drop(['r_useful', 'r_id'], axis=1).values, df_val['r_useful'].values
X_test, y_test = df_test.drop(['r_useful', 'r_id'], axis=1).values, df_test['r_useful'].values

# standardise the features
ss = StandardScaler()
X_train = ss.fit_transform(X_train)
X_val = ss.transform(X_val)
X_test = ss.transform(X_test)

In [8]:
def objective(trial):
  """ Function to tune parameters """
  gc.collect()
  params = {
      "n_estimators":trial.suggest_int('n_estimators', 1, 1000),
      "max_depth" : trial.suggest_int("max_depth", 2, 20),
      "learning_rate" : trial.suggest_float('lr', 1e-2, 3e-1),
  }

  print("Currently running with:")
  print(params)

  model = xgb.XGBRegressor(objective="reg:squarederror",
                            n_jobs=-1,
                            grow_policy='lossguide',
                            tree_method="gpu_hist",
                            predictor="gpu_predictor",
                            booster='gbtree',
                            sampling_method='gradient_based',
                            eval_metrics=['rmse'], 
                            random_state=RANDOM_SEED,
                            enable_categorical=False,
                            early_stopping_rounds=100,
                            **params)
  
  model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)
  y_pred = model.predict(X_val)

  return mean_squared_error(y_val, y_pred, squared=False)

study = opt.create_study(direction='minimize', sampler=TPESampler(seed=RANDOM_SEED))
study.optimize(objective, n_trials=50)
study.best_params

[I 2022-09-23 07:15:17,825] A new study created in memory with name: no-name-90d42e77-4305-4de6-b4c5-8d1c1d4f8566


Currently running with:
{'n_estimators': 436, 'max_depth': 2, 'learning_rate': 0.16940211858482565}


[I 2022-09-23 07:15:22,515] Trial 0 finished with value: 3.6350085188926378 and parameters: {'n_estimators': 436, 'max_depth': 2, 'lr': 0.16940211858482565}. Best is trial 0 with value: 3.6350085188926378.


Currently running with:
{'n_estimators': 436, 'max_depth': 9, 'learning_rate': 0.10579709809112349}


[I 2022-09-23 07:15:38,687] Trial 1 finished with value: 3.3277683600149843 and parameters: {'n_estimators': 436, 'max_depth': 9, 'lr': 0.10579709809112349}. Best is trial 1 with value: 3.3277683600149843.


...

Currently running with:
{'n_estimators': 361, 'max_depth': 13, 'learning_rate': 0.08306189615616857}


[I 2022-09-23 10:27:22,860] Trial 48 finished with value: 3.3451023834888325 and parameters: {'n_estimators': 361, 'max_depth': 13, 'lr': 0.08306189615616857}. Best is trial 36 with value: 3.3032036377549625.


Currently running with:
{'n_estimators': 772, 'max_depth': 10, 'learning_rate': 0.03990043913833198}


[I 2022-09-23 10:28:02,263] Trial 49 finished with value: 3.3306879761483645 and parameters: {'n_estimators': 772, 'max_depth': 10, 'lr': 0.03990043913833198}. Best is trial 36 with value: 3.3032036377549625.


{'n_estimators': 729, 'max_depth': 12, 'lr': 0.035057564389710194}

In [10]:
model = xgb.XGBRegressor(objective="reg:squarederror",
                          n_jobs=-1,
                          grow_policy='lossguide',
                          tree_method="gpu_hist",
                          predictor="gpu_predictor",
                          booster='gbtree',
                          sampling_method='gradient_based',
                          eval_metrics=['rmse'],
                          random_state=RANDOM_SEED,
                          enable_categorical=False,
                          early_stopping_rounds=100,
                          **study.best_params)
model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)

XGBRegressor(early_stopping_rounds=100, enable_categorical=False,
             eval_metrics=['rmse'], grow_policy='lossguide',
             lr=0.035057564389710194, max_depth=12, n_estimators=729, n_jobs=-1,
             objective='reg:squarederror', predictor='gpu_predictor',
             random_state=2, sampling_method='gradient_based',
             tree_method='gpu_hist')

In [11]:
train_pred = model.predict(X_train)
test_pred = model.predict(X_test)
print(f"train results - RMSE: {mean_squared_error(y_train, train_pred, squared=False)}, MAE: {mean_absolute_error(y_train, train_pred)}")
print(f"test results - RMSE: {mean_squared_error(y_test, test_pred, squared=False)}, MAE: {mean_absolute_error(y_test, test_pred)}")

train results - RMSE: 1.7048513955227107, MAE: 1.0215062964365695
test results - RMSE: 3.7631706447347137, MAE: 1.5170065674559365


In [12]:
mean_train = np.mean(y_train)
print(f"Baseline mean model - test RMSE: {np.sqrt(np.mean((y_test - mean_train)**2))}")
print(f"Baseline mean model - test MAE: {np.mean(np.abs(y_test - mean_train))}")

Baseline mean model - test RMSE: 4.312942842075539
Baseline mean model - test MAE: 1.9075376592590827
